In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_PROJECT = os.getenv("LANGCHAIN_PROJECT")
LANGSMITH_TRACING="true"
LANGSMITH_ENDPOINT="https://api.smith.langchain.com"
LANGSMITH_API_KEY= os.getenv("LANGSMITH_API_KEY")

In [3]:
from langchain_groq import ChatGroq
llm = ChatGroq(model_name="Gemma2-9b-It")

In [4]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

d:\learning\learn_langgraph_ss\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from langchain.agents import tool
@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)

In [6]:
from langchain_community.tools import DuckDuckGoSearchResults

In [7]:
search = DuckDuckGoSearchResults()

In [8]:
search.invoke("What is the weather in Bengaluru?")

DuckDuckGoSearchException: https://html.duckduckgo.com/html 202 Ratelimit

In [8]:
tools  = [search]

In [9]:
from langchain import hub

In [10]:
prompt = hub.pull("hwchase17/openai-functions-agent")

In [11]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [12]:
from langchain.agents import create_tool_calling_agent
agent = create_tool_calling_agent(llm, tools, prompt)

In [13]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [14]:
agent_executor.invoke({"input": "What is the weather in Delhi?"})



> Entering new AgentExecutor chain...

Invoking: `duckduckgo_results_json` with `{'query': 'weather in Delhi'}`




DuckDuckGoSearchException: https://lite.duckduckgo.com/lite/ 202 Ratelimit

In [15]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [16]:
loader = WebBaseLoader("https://blog.langchain.dev/langgraph-studio-the-first-agent-ide/")
docs = loader.load()

In [17]:
docs

[Document(metadata={'source': 'https://blog.langchain.dev/langgraph-studio-the-first-agent-ide/', 'title': 'LangGraph Studio: The first agent IDE', 'description': 'LangGraph Studio provides a specialized agent IDE for visualizing, interacting with, and debugging complex agentic applications. See how to use it on your desktop today.', 'language': 'en'}, page_content="\n\n\nLangGraph Studio: The first agent IDE\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nCase Studies\n\n\n\n\nIn the Loop\n\n\n\n\nLangChain\n\n\n\n\nDocs\n\n\n\n\nChangelog\n\n\n\n\n\nSign in\nSubscribe\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nLangGraph Studio: The first agent IDE\nLangGraph Studio provides a specialized agent IDE for visualizing, interacting with, and debugging complex agentic applications. See how to use it on your desktop today.\n\n4 min read\nAug 1, 2024\n\n\n\n\n\nLLMs have paved the way for the dev

In [18]:
documents = RecursiveCharacterTextSplitter(
    chunk_size = 1000, chunk_overlap=200
).split_documents(docs)

In [19]:
vector = FAISS.from_documents(documents, embeddings)
retriever = vector.as_retriever()

In [20]:
retriever.invoke("What is langgraph studio?")

[Document(id='91d9e3e8-fbd3-4d11-9cf9-3262a64f4926', metadata={'source': 'https://blog.langchain.dev/langgraph-studio-the-first-agent-ide/', 'title': 'LangGraph Studio: The first agent IDE', 'description': 'LangGraph Studio provides a specialized agent IDE for visualizing, interacting with, and debugging complex agentic applications. See how to use it on your desktop today.', 'language': 'en'}, page_content="0:00\n\n                            /0:20\n\n\n1×\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nConclusionBuilding agentic applications differs from traditional software development. While code editors remain important, new IDEs designed for agents are also needed. LangGraph Studio is a step in this direction, and we're excited to see how it enhances your workflow.For more on LangGraph Studio, check out our documentation. You can also watch a video walkthrough on YouTube if that's more your style. You can\xa0sign up for LangSmith\xa0today to try out LangGraph Studio for free.We'd also love your fee

In [21]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(
    retriever,
    "langgraph_studio",
    "Search for information about Langgraph. For any questions about Langgraph, you must use this tool!",
)

In [22]:
tools = [search, retriever_tool]

In [23]:
from langchain.agents import create_tool_calling_agent

agent = create_tool_calling_agent(llm, tools, prompt)

In [24]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [25]:
agent_executor.invoke({"input":"Hi! What is a langgraph studio and how to use it?"})



> Entering new AgentExecutor chain...

Invoking: `langgraph_studio` with `{'query': 'What is a langgraph studio and how to use it?'}`


is needed to make the development process easier. Building LLM applications differs from traditional software development, requiring different tooling outside of the traditional code editor.Coding is still important to developing LLM applications — after all, production-ready LangGraph applications have complicated custom logic in the nodes and edges of the graphs that are created. We don't aim to replace code editors but, instead, to augment the development experience with tools tailored for LangGraph applications. LangGraph Studio facilitates this by making it easy to visualize and interact with agent graphs, even if development still primarily happens in code. Visualizing graphs helps developers understand their structure. Furthermore, you can modify an agent result (or the logic underlying a specific node) halfway through the agent's trajectory. 

DuckDuckGoSearchException: https://lite.duckduckgo.com/lite/ 202 Ratelimit

### ReAct Agent (Reasoning + Acting):

**Definition:** The ReAct framework combines reasoning and acting in a single loop to handle tasks. The agent uses natural language reasoning (thinking through steps) and task actions (performing tasks like calculations or data retrieval). The agent uses a combination of reasoning steps to guide actions in real-time, using feedback from those actions to further inform the next step in reasoning.

**How it works:**
- Step 1: The agent receives a question or task.
- Step 2: It reasons aloud (in natural language) about how to solve it.
- Step 3: Based on its reasoning, it takes actions (e.g., searching a database, caclulating something)
- Step 4: The results of these actions are integrated into its reasoning and may trigger further actions.
- Step 5: It repeats the process until it arrives at a solution.

In [27]:
from langchain.utilities import GoogleSerperAPIWrapper
from langchain.agents import Tool
from langchain_core.prompts import PromptTemplate
from langchain.agents import AgentExecutor, create_react_agent

In [28]:
google_search = GoogleSerperAPIWrapper()
tools = [
    Tool(
        name = "Web Answer",
        func = google_search.run,
        description= "Useful for when you need to ask with search",
        verbose = True
    )
]

In [29]:
template = """Answer the following questions as best you can. You have access to the following tools:
{tools}
Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know  the final answer
Final Answer: the final answer to  the original input question
Begin!
Question: {input}
Thought: {agent_scratchpad}"""

In [30]:
prompt = PromptTemplate.from_template(template)

In [31]:
search_agent = create_react_agent(llm, tools, prompt)

In [32]:
agent_executor = AgentExecutor(
    agent = search_agent,
    tools=tools,
    verbose=True,
    return_intermediate_steps=True,
)

In [33]:
agent_executor.invoke({"input":"Where is the hometown of the 2007 T20 Cricket World Cup caption winner and his score"})



> Entering new AgentExecutor chain...
Thought: I need to find out who won the 2007 T20 Cricket World Cup and where they are from. I also need to find their score. 
Action: Web Answer
Action Input: Who won the 2007 T20 Cricket World Cup and their score?Final, Johannesburg, September 24, 2007, ICC World Twenty20, IND Flag IND 157/5, PAK Flag PAK (19.3/20 ov, T:158) 152, India won by 5 runs. India won the tournament, beating Pakistan in the final at Johannesburg. 2007 ICC World Twenty20. Tournament logo. Dates, 11 – ... ... won by 6 wickets (with 12 balls remaining) · 6th Match, Group B • Cape Town, Newlands. england. ENGENG. 188-9 (20). zimbabwe. ZIMZIM. 138-7 (20). England won by ... South Africa won by 8 wickets (with 14 balls remaining). Wed, 12 Sep '07. RESULT. 2nd Match, Group C • Durban, September 12, 2007, ICC World Twenty20. India won by defeating Pakistan by five runs. The teams had previously played each other in a Group-D match of the tournament, which was also won by India.

{'input': 'Where is the hometown of the 2007 T20 Cricket World Cup caption winner and his score',
 'output': 'New Delhi',
 'intermediate_steps': [(AgentAction(tool='Web Answer', tool_input='Who won the 2007 T20 Cricket World Cup and their score?', log='Thought: I need to find out who won the 2007 T20 Cricket World Cup and where they are from. I also need to find their score. \nAction: Web Answer\nAction Input: Who won the 2007 T20 Cricket World Cup and their score?'),
   'Final, Johannesburg, September 24, 2007, ICC World Twenty20, IND Flag IND 157/5, PAK Flag PAK (19.3/20 ov, T:158) 152, India won by 5 runs. India won the tournament, beating Pakistan in the final at Johannesburg. 2007 ICC World Twenty20. Tournament logo. Dates, 11 – ... ... won by 6 wickets (with 12 balls remaining) · 6th Match, Group B • Cape Town, Newlands. england. ENGENG. 188-9 (20). zimbabwe. ZIMZIM. 138-7 (20). England won by ... South Africa won by 8 wickets (with 14 balls remaining). Wed, 12 Sep \'07. RESULT. 

In [34]:
agent_executor.invoke({"input":"Who is a god of cricket what is highest score of the god of cricket."})



> Entering new AgentExecutor chain...
Thought: This question seems to be a bit of a trick! There isn't an official "god of cricket." It's likely a playful way of referring to a legendary or exceptionally skilled cricketer. 

To find out who people consider the "god of cricket," I'll need to search the web.
Action: Web Answer
Action Input: Who is considered the god of cricket?Sachin Tendulkar. Fondly called the “God of Cricket” or “Master Blaster,” Sachin Tendulkar is considered one of the greatest batsmen of all time in the sport that is considered a national institution in India, cricket.Thought: I now know who is considered the god of cricket. Now I need to find Sachin Tendulkar's highest score. 
Action: Web Answer
Action Input: What is Sachin Tendulkar's highest score in cricket?Sachin Tendulkar: Cricketer and former Member of Rajya Sabha. Sachin Ramesh Tendulkar is an Indian former international cricketer who captained the Indian national team. Sachin Tendulkar Born: April 24, 19

{'input': 'Who is a god of cricket what is highest score of the god of cricket.',
 'output': 'Sachin Tendulkar is often called the "God of Cricket." His highest score is 248 not out, which he achieved in a Test match against Bangladesh in 2004.',
 'intermediate_steps': [(AgentAction(tool='Web Answer', tool_input='Who is considered the god of cricket?', log='Thought: This question seems to be a bit of a trick! There isn\'t an official "god of cricket." It\'s likely a playful way of referring to a legendary or exceptionally skilled cricketer. \n\nTo find out who people consider the "god of cricket," I\'ll need to search the web.\nAction: Web Answer\nAction Input: Who is considered the god of cricket?'),
   'Sachin Tendulkar. Fondly called the “God of Cricket” or “Master Blaster,” Sachin Tendulkar is considered one of the greatest batsmen of all time in the sport that is considered a national institution in India, cricket.'),
  (AgentAction(tool='Web Answer', tool_input="What is Sachin Te

### ReAct with Custom Tool

In [35]:
@tool
def get_employee_id(name):
    """
    To get employee id, it takes employee name as arguments
    name(str): Name of the employee
    """
    fake_employee = {
        "Alice":"E001",
        "Bob":"E002",
        "Charlie":"E003",
        "Diana":"E004",
        "Evan":"E005",
        "Fiona":"E006",
        "George":"E007",
        "Hannah":"E008",
        "Ian":"E009",
        "Jasmine":"E010",
    }
    
    return fake_employee.get(name, "Employee not found")

In [36]:
@tool
def get_employee_salary(employee_id):
    """
    To get the salary of an employee, it takes employee_id as input and return salary
    """
    employee_salaries = {
        "E001": 56000,
        "E002": 47000,
        "E003": 52000,
        "E004": 61000,
        "E005": 45000,
        "E006": 58000,
        "E007": 49000,
        "E008": 53000,
        "E009": 50000,
        "E010": 55000
    }
    return employee_salaries.get(employee_id, "Employee not found")

In [37]:
prompt = hub.pull("hwchase17/react")

In [38]:
print(prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [39]:
tools = [get_employee_salary, get_employee_id]

In [40]:
agent = create_react_agent(llm, tools, prompt)

In [41]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [42]:
agent_executor.invoke({"input": "What is the salary of Evan?"})



> Entering new AgentExecutor chain...
Thought: I need to find Evan's employee ID first, then use that to get his salary.
Action: get_employee_id
Action Input: EvanE005Thought: I can now use Evan's employee ID to get his salary. 
Action: get_employee_salary
Action Input: E00545000Thought: I now know the final answer
Final Answer: Evan's salary is 45000.  


> Finished chain.


{'input': 'What is the salary of Evan?', 'output': "Evan's salary is 45000."}

In [43]:
agent_executor.invoke({"input": "What is the salary of Sunny?"})



> Entering new AgentExecutor chain...
Thought: I need to find Sunny's employee ID first.
Action: get_employee_id
Action Input: SunnyEmployee not foundThought:  It seems Sunny is not in the system. 
Final Answer:  Employee not found 


> Finished chain.


{'input': 'What is the salary of Sunny?', 'output': 'Employee not found'}

### Self-Ask with Search Agent:
**Definitions:**
- This approach allows the AI to ask itself follow-up questions when it doesn't immediediately know the answer. It performs a recursive questioning process to break down complex queries into smaller, more manageable ones.
- Once the AI generates these follow-up questions, it performs an external search (e.g., through Google or an internal database) to gather the necessary information before formulating a response.

**How it works:**
- Step 1: Receive a complex question
- Step 2: The agent identifies sub-questions or follow-up questions.
- Step 3: It performs a search or fetches answers to these questions from external resources (like ).
- Step 4: The answers are aggregated to provide a complete response.

**Key points:**
- Emphasizes question decomposition
- Relies on external search for sub-questions.
- Useful for answering open-ended or broad questions where the answer is not immediately available.

In [44]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [45]:
from langchain_groq import ChatGroq
llm = ChatGroq(model_name="Gemma2-9b-It")

In [46]:
from langchain import hub
from langchain.agents import AgentExecutor, create_self_ask_with_search_agent
from langchain_community.tools import DuckDuckGoSearchResults

In [47]:
tools = [
    Tool(
        name = "Intermediate Answer",
        func = google_search.run,
        description="useful for when you need to ask",
        verbose = True
    )
]

In [48]:
prompt = hub.pull("hwchase17/self-ask-with-search")

In [49]:
print(prompt.template)

Question: Who lived longer, Muhammad Ali or Alan Turing?
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali

Question: When was the founder of craigslist born?
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952

Question: Who was the maternal grandfather of George Washington?
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washin

In [50]:
agent = create_self_ask_with_search_agent(llm, tools, prompt)

In [51]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

In [52]:
agent_executor.invoke({"input":"Where is the hometown of the current prime minister of India?"})



> Entering new AgentExecutor chain...
Yes.


Follow up: Who is the current prime minister of India?Narendra ModiCould not parse output: Follow up: Where is Narendra Modi from? 


For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE Invalid or incomplete responseCould not parse output: The issue you're encountering is related to LangChain's inability to correctly parse the output from your tool (likely a search engine or API) and extract the relevant information. 

Here's a breakdown of the problem and potential solutions:

**Problem:**

* **Inconsistent Output:**  Search engines and APIs often provide responses in various formats (paragraphs, lists, structured data). LangChain needs a consistent structure to extract the answer accurately.
* **Missing Keywords:**  When you ask "Where is Narendra Modi from?", the tool might return a complex response about his biography, not directly stating his hometown. LangChain might miss the ke

{'input': 'Where is the hometown of the current prime minister of India?',
 'output': 'Agent stopped due to iteration limit or time limit.'}